![header](https://i.imgur.com/I4ake6d.jpg)

# IN SITU GLOBAL SEAS TRAINING

<div style="text-align: right"><i> 13-05-Part-five-out-of-five </i></div>

***
# GLO `NRT` product/dataset: managing files (gliders)

***
**General Note 1**: Execute each cell through the <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button from the top MENU (or keyboard shortcut `Shift` + `Enter`).<br>
<br>
**General Note 2**: If, for any reason, the kernel is not working anymore, in the top MENU, click on the <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Then, in the top MENU, click on "Cell" and select "Run All Above Selected Cell".<br>
***

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#1.-Introduction" data-toc-modified-id="1.-Introduction">1. Introduction</a></span></li>
        <li>
            <span><a href="#2.-Setup" data-toc-modified-id="2.-Setup">2. Setup</a></span>
            <ul>
                <li><span><a href="#2.1.-Python-packages" data-toc-modified-id="2.1.-Python-packages">2.1. Python packages</a></span></li>
            </ul>
        </li>
        <li><span><a href="#3.-Profilers-(PF)-data" data-toc-modified-id="3.-Profilers-(PF)-data">3. Profilers (PF) data</a></span>
            <ul>
                <li><span><a href="#3.1.-Reading-the-file" data-toc-modified-id="3.1.-Reading-the-file">3.1. Reading the file</a></span></li>
                <li><span><a href="#3.2.-Data-visualization" data-toc-modified-id="3.2.-Data-visualization">3.2. Data visualization</a></span>
                    <ul>
                        <li><span><a href="#3.2.1.-Trajectory-animation" data-toc-modified-id="3.2.1.-Trajectory-animation">3.2.1. Trajectory animation</a></span></li>
                        <li><span><a href="#3.2.2.-1D-profile" data-toc-modified-id="3.2.2.-1D-profile">3.2.2. 1D profile</a></span></li>
                        <li><span><a href="#3.2.3.-2D-profile" data-toc-modified-id="3.2.3.-2D-profile">3.2.3. 2D profile</a></span></li>
                        <li><span><a href="#3.2.4.-3D-profile" data-toc-modified-id="3.2.4.-3D-profile">3.2.4. 3D profile</a></span></li>
                    </ul>
                </li>
           </ul>
        </li>
        <li><span><a href="#4.-Wrap-up" data-toc-modified-id="4.-Wrap-up">4. Wrap-up</a></span></li>
    </ul>
</div>

***

## 1. Introduction
[Go back to the "Table of Contents"](#Table-of-Contents)

According to the [13-01-NearRealTtime-product-collections-overview.ipynb](13-01-NearRealTtime-product-collections-overview.ipynb) one of the data source types available are Gliders. Please use the notebook [13-02-NearRealTtime-product-subsetting-download](13-02-NearRealTtime-product-subsetting-download.ipynb) to download some files from Profilers ('GL' data type) and let's check its data. If you wanna skip the downloading part you can use the netCDF files available `/data/nc_files/GL` instead.  

## 2. Setup
[Go back to the "Table of Contents"](#Table-of-Contents)

### 2.1. Python packages

For the notebook to properly run we need to first load the next packages available from the Jupyter Notebook Ecosystem. Please run the `next cell`:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import IPython
import datetime
import pandas as pd
import os
import numpy as np
import xarray
import matplotlib.pyplot as plt
from matplotlib import colors
import folium
from folium import plugins
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import YouTubeVideo
%matplotlib inline

<div class="alert alert-block alert-warning">
<b>WARNING</b>
    
***  
If any of them raises any error it means you need to install the module first. For doing so please:
1. Open a new cell in the notebook
2. Run <i>`!conda install packageName --yes`</i> or <i>`!conda install -c conda-forge packageName --yes`</i> or <i>`!pip install packageName`</i>
3. Import again!
<br><br>
Example: <i>how-to-solve import error for json2html module </i>

![region.png](img/errorImporting.gif)

### 2.1. Auxiliary functions

Please `run the next cell` to load into memory a functions we will use later on for subsetting the original file if it results too large to fit int memmory:

In [ ]:
def get_subset(start,end,ds):
    #Subsets a dataset (ds) from start to end dates
    i_start = ds['TIME'].astype(str).values.tolist().index(ds['TIME'][ds['TIME'].astype(str).str.contains(start) == True].astype(str).values[0])
    i_end = ds['TIME'].astype(str).values.tolist().index(ds['TIME'][ds['TIME'].astype(str).str.contains(end) == True].astype(str).values[-1])
    return ds.isel(TIME=slice(i_start, i_end),LATITUDE=slice(i_start, i_end),LONGITUDE=slice(i_start, i_end),POSITION=slice(i_start, i_end))

## 3. Gliders (GL) data
[Go back to the "Table of Contents"](#Table-of-Contents)

Glider area autonomous vehicles remotely piloted that are released into the ocean in other to measure a number of variables such as temperature and salinity. This platform performs a set of up/downs casts along the water column to retrieve the variation of such parameters along the depth (profiles!) by regulatings its buoyancy.

<br>Here a self explanatory video from SOCIB:  `run the next cell`

In [ ]:
YouTubeVideo('mhYVcmT06X0', width="100%", height=500)

Let's see the data of one of the available gliders in the GLO Seas.

### 3.1. Reading the file

`Run the next cell` to see the profilers files already available in the /data folder:

In [ ]:
dir_GL = os.path.join(os.getcwd(),'data','nc_files','GL') 
os.listdir(dir_GL)

`Set one` of the above available `file name` and `run the next cells`:

In [ ]:
file = 'GL_PR_GL_6801636.nc'
path = os.path.join(dir_GL, file)

In [ ]:
ds = xarray.open_dataset(path)
ds.close()
ds

The above one is an overview of the content of the file: variables, dimensions, coordinates, global attributes...
<br>Let's list now the available variables: `run the next cell`

In [ ]:
for var in ds.variables:
    print(var + ':' + ds[var].attrs['long_name'])

### 3.2. Data visualization

Let's focus on one of the above variables to visualize its data!: `set one and run the next cell`

In [ ]:
param = 'TEMP'

In [ ]:
ds[param]

#### 3.2.1. Trajectory animation

As stated before, the profilers are mobile platforms. <br>Let's check the overall trajectory by joining the sampling points (locations where a profile has been performed).

In In Situ TAC netCDFs all variables are linked to another called the same plus '_QC'. This 'twin' variable contains a quality flag for each value in the paired variable.`run the next cell` to check the flag values convention:

In [ ]:
pd.DataFrame(data=ds['POSITION_QC'].attrs['flag_values'],
             index=ds['POSITION_QC'].attrs['flag_meanings'].split(' '), columns=['quality flag'])

Users are recommended to use only the data flagged as 1, they so called 'good data'. Let's then check the available flags for the coordinates to see if we need to get rid of not-good values: `run the next cells`

In [ ]:
ds['POSITION_QC'].plot(aspect=2, size=5)

Above, we might detect some flags values different from 1 (or not!). As a general rule, we recommend to make a selection of the data to only work with 'good' data (flags = 1). See next how:`run the next cell`

In [ ]:
lats = ds['LATITUDE'].where(ds['POSITION_QC'] == 1).values.tolist()
lats = [i[0] for i in lats]
lons = ds['LONGITUDE'].where(ds['POSITION_QC'] == 1).values.tolist()
lons = [i[1] for i in lons]
times = ds['TIME'].values.tolist()
strtimes = ds['TIME'].values[:]

Let's initialize now a geojson object that will represent our glider: `run the next cell`

In [ ]:
glider = {
    'type': 'Feature',
    'geometry': {
        'type': 'LineString',
        'coordinates': []#to be populated with lat,lon pairs
    },
    'properties': {
        'times': [] #to be populed with time strings
    }
}

Let's now popule it: `run the next cell`

In [ ]:
for time, strtime, lat, lon in zip(times, strtimes, lats, lons):
    base = [time,lat,lon]
    if(any(x is None for x in base)):
        continue
    if(any(np.isnan(x) for x in base)):
        continue
    glider['properties']['times'].append(str(strtime)[:22])
    glider['geometry']['coordinates'].append([lon, lat])

Let's now create a map to finally see the overall trajectory performed by the profiler: `run the next cell`

In [ ]:
mean_lat, mean_lon = np.mean(lats), np.mean(lons)
m = folium.Map(
    location=[mean_lat, mean_lon], zoom_start=5,
    #tiles='https://tiles.emodnet-bathymetry.eu/2020/baselayer/web_mercator/{z}/{x}/{y}.png',attr="EMODnet bathy"
)
marker = plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': [glider],
}, period='P1D', add_last_point=True, loop=False).add_to(m)
m

<div class="alert alert-block alert-warning">
<b>WARNING</b>
    
***  
If you do not see any map when running the next cell please change your navigator (try chrome!).

Let's make a subset: from Madeira to Canary Islands!

In [ ]:
subset = get_subset('2020-03-29','2020-04-20',ds)

#### 3.2.2. 1D profile

Let's see the selected variable variation with depth in a specific time/position..

Set first cell the profile to check (`run the next cell`): 

In [ ]:
idx = 0 #First profile

Set a title for the plot (`run the next cell`):

In [ ]:
title = param + ' profile from platform '+subset.platform_code+\
    ' \n time('+str(subset['TIME'].values[idx])[:19]+') '+\
    '& position('+str(subset['LATITUDE'].values[idx])+','+str(subset['LONGITUDE'].values[idx])+')'

Create plot (`run the next cell`): *uncomment the last to lines to save the output as png

In [ ]:
subset[param][idx].where(subset[param+'_QC'][idx]==1).plot.line(aspect=2, size=7, marker='o',y='DEPTH')
plt.title(title)
plt.gca().invert_yaxis()
#png_name ='1D_'+ds.platform_code+'.png'
#plt.savefig(png_name)

#### 3.2.3. 2D profile

Let's plot the selected variable variation with time and depth (no matter the position)!

Set a new title (`run the next cell`):

In [ ]:
title = '2D '+param+' profiles from platform '+subset.platform_code+\
    ' \n times('+str(subset['TIME'].values[0])[:19]+' / '+str(subset['TIME'].values[-1])[:19]+') '

Create a 2D plot (`run the next cell`): *uncomment the last to lines to save the output as png

In [ ]:
subset[param].where(subset[param+'_QC'] == 1).plot(
    x='TIME', y='DEPTH', aspect=3, size=10, cmap='jet')
plt.title(title)
plt.gca().invert_yaxis()
#png_name ='2D_'+ds.platform_code+'.png'
#plt.savefig(png_name)

#### 3.2.4. 3D profile

Let's plot the selected variable variation with time, depth and position!

Set first a colormap (`run the next cell`):

In [ ]:
cmap = plt.cm.Spectral_r
norm = colors.Normalize(vmin=ds[param].where(subset[param+'_QC'] == 1).min(
).values.tolist(), vmax=subset[param].where(subset[param+'_QC'] == 1).max().values.tolist())

Specify the list of profiles to plot (`run the next cell`):

In [ ]:
profiles2plot = range(0, len(subset['TIME'])) #Defaults to all!
profiles2plot

<div class="alert alert-block alert-warning">
<b>WARNING</b>
    
***  
If you face any problem with the next plot, try reducing the number of `profiles2plot`

Create the 3D plot (`run the next cell`): *uncomment the last to lines to save the output as png

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
for a in profiles2plot:
    lat = subset['LATITUDE'].values.tolist()[a]
    lon = subset['LONGITUDE'].values.tolist()[a]
    temp = subset[param].where(subset[param+'_QC'] == 1).values.tolist()[a]
    depth = (-1*subset['DEPTH']).values.tolist()
    plt.scatter(lon*np.ones(len(depth)), lat*np.ones(len(depth)), zs=depth, zdir='z', s=20, c=temp, cmap=cmap, norm=norm)
cbar = plt.colorbar(orientation="horizontal", pad=0.02)
cbar.ax.set_xlabel(param)
ax.set_title(str(a+1)+param+ ' profiles from platform ' + ds.id, y=1.08)
ax.set_zlabel('depth', labelpad=20, rotation=90)
ax.set_ylabel('latitude', labelpad=20)
ax.set_xlabel('longitude', labelpad=20)
#png_name ='3D_'+ds.platform_code+'.png'
#plt.savefig(png_name)

***

## 4. Wrap-up
[Go back to the "Table of Contents"](#Table-of-Contents)

So far you should already know how to deal with profile-like data from profilers platforms. <br> `If you don't please ask us! it is the moment!`